In [1]:
import os
from pathlib import Path
from collections import namedtuple
from itertools import starmap
import numpy as np

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day18.txt'

## Part One

In [2]:
class Point(namedtuple("Point", ['x', 'y', 'z'])):

    def __add__(self, other):
        return Point(*(s + o for s, o in zip(self, other)))
    
    def neighbors(self):
        return set(self + offset for offset in self.offsets)
    
    def inside(self, min_extent, max_extent):
        return all(
            _min <= coord <= _max 
            for coord, (_min, _max) 
            in zip(self, zip(min_extent, max_extent))
        )

Point.offsets = list(starmap(Point, (
        (1, 0, 0), (0, 1, 0), (0, 0, 1),
        (-1, 0, 0), (0, -1, 0), (0, 0, -1)
)))
    
with open(FOLDER / in_file) as f:
    cubes = set(starmap(Point, (map(int,line.split(',')) for line in f)))
    
# Count any neighbor that is not in cubes — it represents an exposed face
sum(len(c.neighbors()-cubes) for c in cubes)

4340

# Part Two

**BFS from outsid the blob**

When we bump into a cube, count that face. 

It should not be possible to bump into the same cube from the same direction, so each face only gets counted once.

In [3]:
from collections import deque

min_extent = [min(coord) - 1 for coord in zip(*cubes)]
max_extent = [max(coord) + 1 for coord in zip(*cubes)]

q = deque([Point(*min_extent)])
seen = set()
face_count = 0

while q:
    current = q.popleft()
    for n in current.neighbors():
        if n.inside(min_extent, max_extent):
            if n in cubes:
                face_count += 1
            elif n not in seen:
                q.append(n)
                seen.add(n)


face_count

2468